In [1]:
!pip install pyomo

You should consider upgrading via the '/Users/fpasten/Uni/CINF105/sol1/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from pyomo.environ import *

# Problema 1

In [62]:
# Creando modelo base
problem_1 = ConcreteModel()

# set T 1..4
problem_1.T = RangeSet(1,4)

# set I 1..3
problem_1.I= RangeSet(1,3)

face_value = {1: 120000, 2: 200000, 3: 1000000}

# param C{I}
problem_1.C = Param(problem_1.I, initialize=face_value)

coupon_rate = {1: 0.05, 2: 0.035, 3: 0.12}

# param TC{I}
problem_1.TC = Param(problem_1.I, initialize=coupon_rate)

# var Tengo{T union {0}} >= 0
problem_1.Tengo = Var(problem_1.T.union({0}), domain=NonNegativeReals)

# var Sobra{T} >= 0
problem_1.Sobra = Var(problem_1.T, domain=NonNegativeReals)

# var x{I,T} >= 0 integer
problem_1.x = Var(problem_1.I, problem_1.T, domain=NonNegativeIntegers)

# maximize z: Tengo[4]
problem_1.z = Objective(expr=problem_1.Tengo[4], sense=maximize)

# Define the constraints
problem_1.rPres = ConstraintList()
for t in problem_1.T:
    problem_1.rPres.add(problem_1.Sobra[t] == problem_1.Tengo[t-1] - sum(problem_1.x[i,t]*problem_1.C[i] for i in problem_1.I))

problem_1.rini1 = Constraint(expr=problem_1.Tengo[0] == 10000000)
problem_1.r1 = Constraint(expr=problem_1.Tengo[1] == problem_1.Sobra[1] + problem_1.x[1,1]*problem_1.TC[1]*problem_1.C[1])
problem_1.r2 = Constraint(expr=problem_1.Tengo[2] == problem_1.Sobra[2] + problem_1.x[1,1]*problem_1.TC[1]*problem_1.C[1] +
                            problem_1.x[1,2]*problem_1.TC[1]*problem_1.C[1] + problem_1.x[2,2]*problem_1.TC[2]*problem_1.C[2])
problem_1.r3 = Constraint(expr=problem_1.Tengo[3] == problem_1.Sobra[3] + problem_1.x[1,1]*(1+problem_1.TC[1])*problem_1.C[1] +
                            problem_1.x[1,2]*problem_1.TC[1]*problem_1.C[1] + problem_1.x[2,2]*(1+problem_1.TC[2])*problem_1.C[2] +
                            problem_1.x[2,3]*problem_1.TC[2]*problem_1.C[2] + problem_1.x[3,3]*(1+problem_1.TC[3])*problem_1.C[3])
problem_1.r4 = Constraint(expr=problem_1.Tengo[4] == problem_1.Sobra[4] + problem_1.x[1,2]*(1+problem_1.TC[1])*problem_1.C[1] +
                            problem_1.x[2,3]*(1+problem_1.TC[2])*problem_1.C[2] + problem_1.x[3,4]*(1+problem_1.TC[3])*problem_1.C[3])


# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_1)

# Imprimiendo resultados
print('Objective function value: ', problem_1.z())
for i in problem_1.I:
    for t in problem_1.T:
        print('x[',i,',',t,'] = ', problem_1.x[i,t]())

Objective function value:  12934000.0
x[ 1 , 1 ] =  83.0
x[ 1 , 2 ] =  0.0
x[ 1 , 3 ] =  -0.0
x[ 1 , 4 ] =  0.0
x[ 2 , 1 ] =  0.0
x[ 2 , 2 ] =  0.0
x[ 2 , 3 ] =  0.0
x[ 2 , 4 ] =  -0.0
x[ 3 , 1 ] =  -0.0
x[ 3 , 2 ] =  -0.0
x[ 3 , 3 ] =  1.0
x[ 3 , 4 ] =  11.0


# variación propuesta

todos los bonos disponibles desde el primer mes

# Problema 2

In [71]:
problem_2 = ConcreteModel()

# set I 1,2,3
problem_2.I = RangeSet(1, 3)

# set J 1,2
problem_2.J = RangeSet(1, 3)

#contribution dictionary
contrib = {(1,1): 15, (1,2): 8, (1,3): 12, 
           (2,1): 3, (2,2): 2, (2,3): 1, 
           (3,1): 4, (3,2): 5, (3,3): 7}

# param a{I,J}
problem_2.a = Param(problem_2.I, problem_2.J, initialize=contrib)

goal = {1: 1000000, 2: 1000, 3: 100}

# param meta{I}
problem_2.meta = Param(problem_2.I, initialize=goal)

# Define the variables
problem_2.x = Var(problem_2.J, within=NonNegativeReals)
problem_2.yma = Var(problem_2.I, within=NonNegativeReals)
problem_2.yme = Var(problem_2.I, within=NonNegativeReals)

# Define the objective function
problem_2.z = Objective(expr=problem_2.yme[1]*6 + 3*problem_2.yma[2] + 4*problem_2.yme[2] + 4*problem_2.yma[3], sense=minimize)

# Define the constraints
problem_2.rMetas = ConstraintList()
for i in problem_2.I:
    problem_2.rMetas.add(sum(problem_2.x[j] * problem_2.a[i, j] for j in problem_2.J) - problem_2.meta[i] == problem_2.yma[i] - problem_2.yme[i])

# Solve the problem
solver = SolverFactory('gurobi')
results = solver.solve(problem_2)

# Print the optimal solution
print("Objective Value:", problem_2.z())
print("x:")
for j in problem_2.J:
    print(j, problem_2.x[j].value)
print("yma:")
for i in problem_2.I:
    print(i, problem_2.yma[i].value)
print("yme:")
for i in problem_2.I:
    print(i, problem_2.yme[i].value)

Objective Value: 1663266.6666666667
x:
1 66666.66666666667
2 0.0
3 0.0
yma:
1 0.0
2 199000.0
3 266566.6666666667
yme:
1 0.0
2 0.0
3 0.0


# Problema 3

In [4]:
problem_3 = ConcreteModel()

# set T 1..6
problem_3.T = RangeSet(1, 6)

# set M 1..2
problem_3.M = RangeSet(1, 2)

cpu_opcost_perpos = {(1,1,1): 29, (1,2,1): 22, (1,3,1): 7, (1,4,1): 33, (1,5,1): 14, (1,6,1): 34,
                     (2,1,1): 8, (2,2,1): 32, (2,3,1): 14, (2,4,1): 30, (2,5,1): 15, (2,6,1): 27,
                     (3,1,1): 31, (3,2,1): 20, (3,3,1): 18, (3,4,1): 7, (3,5,1): 22, (3,6,1): 29,
                     (4,1,1): 30, (4,2,1): 29, (4,3,1): 25, (4,4,1): 12, (4,5,1): 29, (4,6,1): 7,
                     (5,1,1): 26, (5,2,1): 17, (5,3,1): 12, (5,4,1): 27, (5,5,1): 7, (5,6,1): 9,
                     (6,1,1): 20, (6,2,1): 29, (6,3,1): 12, (6,4,1): 6, (6,5,1): 23, (6,6,1): 32,
                     (1,1,2): 6, (1,2,2): 27, (1,3,2): 35, (1,4,2): 31, (1,5,2): 24, (1,6,2): 19,
                     (2,1,2): 7, (2,2,2): 10, (2,3,2): 7, (2,4,2): 28, (2,5,2): 32, (2,6,2): 29,
                     (3,1,2): 2, (3,2,2): 9, (3,3,2): 17, (3,4,2): 31, (3,5,2): 17, (3,6,2): 27,
                     (4,1,2): 20, (4,2,2): 5, (4,3,2): 19, (4,4,2): 15, (4,5,2): 33, (4,6,2): 33,
                     (5,1,2): 3, (5,2,2): 21, (5,3,2): 7, (5,4,2): 33, (5,5,2): 10, (5,6,2): 28,
                     (6,1,2): 15, (6,2,2): 7, (6,3,2): 19, (6,4,2): 25, (6,5,2): 12, (6,6,2): 21}

# param C{T, T, M}
problem_3.C = Param(problem_3.T, problem_3.T, problem_3.M, initialize=cpu_opcost_perpos)

# var X{T, T, M} >= 0 binary
problem_3.X = Var(problem_3.T, problem_3.T, problem_3.M, domain=Binary)

# define objective function z sum{i in T, j in T, m in M} C[i,j,m] * X[i,j,m]
problem_3.z = Objective(expr=sum(problem_3.C[i,j,m] * problem_3.X[i,j,m] for i in problem_3.T for j in problem_3.T for m in problem_3.M), sense=minimize)

# define constraints
# sum{i in T} X[i,j,m] <= 1 for all j in T, m in M
problem_3.r1 = ConstraintList()
for j in problem_3.T:
    for m in problem_3.M:
        problem_3.r1.add(sum(problem_3.X[i,j,m] for i in problem_3.T) <= 1)

# sum{j in T, m in M} X[i,j,m] = 1 for all i in T
problem_3.r2 = ConstraintList()
for i in problem_3.T:
    problem_3.r2.add(sum(problem_3.X[i,j,m] for j in problem_3.T for m in problem_3.M) == 1)

# sum{i in T} X[i,j,m] <= sum{i in T} X[i,j-1,m] for all j in T-1, m in M
problem_3.r3 = ConstraintList()
for j in problem_3.T:
    if j > 1:
        for m in problem_3.M:
            problem_3.r3.add(sum(problem_3.X[i,j,m] for i in problem_3.T) <= sum(problem_3.X[i,j-1,m] for i in problem_3.T))


# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_3)

# Imprimiendo resultados
print('Objective function value: ', problem_3.z())
print('Solution:')
for i in problem_3.T:
    for j in problem_3.T:
        for m in problem_3.M:
            if problem_3.X[i,j,m]() > 0:
                print('X[', i, ',', j, ',', m, '] = ', problem_3.X[i,j,m]())




Objective function value:  45.0
Solution:
X[ 1 , 3 , 1 ] =  1.0
X[ 2 , 1 , 1 ] =  1.0
X[ 3 , 1 , 2 ] =  1.0
X[ 4 , 2 , 2 ] =  1.0
X[ 5 , 2 , 1 ] =  1.0
X[ 6 , 4 , 1 ] =  1.0


## Variación propuesta 



# Problema 4

In [6]:
problem_4 = ConcreteModel()

# set I 1..2
problem_4.I = RangeSet(1, 2)

# set J 1..4
problem_4.J = RangeSet(1, 4)

# param Gas
problem_4.Gas = Param(initialize=51000)

kmGalon = {1: 3, 2: 4.5}

# Param kmGalon{I}
problem_4.kmGalon = Param(problem_4.I, initialize=kmGalon)

cap = {1: 48, 2: 35}

# param Cap{I}
problem_4.Cap = Param(problem_4.I, initialize=cap)

dis = {1: 675, 2: 720, 3: 810, 4: 900}

# param Dis{J}
problem_4.Dis = Param(problem_4.J, initialize=dis)

failprob = {(1,1): 0.9, (1,2): 0.8, (1,3): 0.85, (1,4): 0.75,
            (2,1): 0.92, (2,2): 0.84, (2,3): 0.88, (2,4): 0.8}

# param pF{I,J}
problem_4.pF = Param(problem_4.I, problem_4.J, initialize=failprob)

# param M
problem_4.M = Param(initialize=1000000)

# var x{I,J} >= 0 integer
problem_4.x = Var(problem_4.I, problem_4.J, domain=NonNegativeIntegers)

# var y{J} binary
problem_4.y = Var(problem_4.J, domain=Binary)

# define objective function z
# minimize sum{i in I, j in J} log(pF[i,j]) * x[i,j]

problem_4.z = Objective(expr=sum(log(problem_4.pF[i,j]) * problem_4.x[i,j] for i in problem_4.I for j in problem_4.J), sense=minimize)

# define constraints
# sum{j in J} x[i,j] <= Cap[i] for all i in I
problem_4.r1 = ConstraintList()
for i in problem_4.I:
    problem_4.r1.add(sum(problem_4.x[i,j] for j in problem_4.J) <= problem_4.Cap[i])

# sum{i in I, j in J} (2 * Dis[j] / kmGalon[i] + 150) * x[i,j] <= Gas
problem_4.r2 = Constraint(expr=sum((2 * problem_4.Dis[j] / problem_4.kmGalon[i] + 150) * problem_4.x[i,j] for i in problem_4.I for j in problem_4.J) <= problem_4.Gas)

# sum{i in I} x[i,j] <= M * y[j] for all j in J
problem_4.r3 = ConstraintList()
for j in problem_4.J:
    problem_4.r3.add(sum(problem_4.x[i,j] for i in problem_4.I) <= problem_4.M * problem_4.y[j])

# sum{i in I} x[i,j] >= y[j] for all j in J
problem_4.r4 = ConstraintList()
for j in problem_4.J:
    problem_4.r4.add(sum(problem_4.x[i,j] for i in problem_4.I) >= problem_4.y[j])

# sum{j in J} y[j] >= 2
problem_4.r5 = Constraint(expr=sum(problem_4.y[j] for j in problem_4.J) >= 2)

# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_4)

# Imprimiendo resultados
print('Objective function value: ', problem_4.z())
for i in problem_4.I:
    for j in problem_4.J:
        print('x[',i,',',j,'] = ', problem_4.x[i,j]())





Objective function value:  -19.924168052908417
x[ 1 , 1 ] =  -0.0
x[ 1 , 2 ] =  -0.0
x[ 1 , 3 ] =  -0.0
x[ 1 , 4 ] =  44.0
x[ 2 , 1 ] =  -0.0
x[ 2 , 2 ] =  2.0
x[ 2 , 3 ] =  -0.0
x[ 2 , 4 ] =  31.0


In [9]:
problem_4 = ConcreteModel()

# set I 1..2
problem_4.I = RangeSet(1, 2)

# set J 1..4
problem_4.J = RangeSet(1, 4)

# param Gas
problem_4.Gas = Param(initialize=51000)

kmGalon = {1: 3, 2: 4.5}

# Param kmGalon{I}
problem_4.kmGalon = Param(problem_4.I, initialize=kmGalon)

cap = {1: 48, 2: 35}

# param Cap{I}
problem_4.Cap = Param(problem_4.I, initialize=cap)

dis = {1: 675, 2: 720, 3: 810, 4: 900}

# param Dis{J}
problem_4.Dis = Param(problem_4.J, initialize=dis)

failprob = {(1,1): 0.9, (1,2): 0.8, (1,3): 0.85, (1,4): 0.75,
            (2,1): 0.92, (2,2): 0.84, (2,3): 0.88, (2,4): 0.8}

# param pF{I,J}
problem_4.pF = Param(problem_4.I, problem_4.J, initialize=failprob)

# param M
problem_4.M = Param(initialize=1000000)

# var x{I,J} >= 0 integer
problem_4.x = Var(problem_4.I, problem_4.J, domain=NonNegativeIntegers)

# var y{J} binary
problem_4.y = Var(problem_4.J, domain=Binary)

# define objective function z
# minimize sum{i in I, j in J} log(pF[i,j]) * x[i,j]

problem_4.z = Objective(expr=sum(log(problem_4.pF[i,j]) * problem_4.x[i,j] for i in problem_4.I for j in problem_4.J), sense=minimize)

# define constraints
# sum{j in J} x[i,j] <= Cap[i] for all i in I
problem_4.r1 = ConstraintList()
for i in problem_4.I:
    problem_4.r1.add(sum(problem_4.x[i,j] for j in problem_4.J) <= problem_4.Cap[i])

# sum{i in I, j in J} (2 * Dis[j] / kmGalon[i] + 150) * x[i,j] <= Gas
problem_4.r2 = Constraint(expr=sum((2 * problem_4.Dis[j] / problem_4.kmGalon[i] + 150) * problem_4.x[i,j] for i in problem_4.I for j in problem_4.J) <= problem_4.Gas)

# sum{i in I} x[i,j] <= M * y[j] for all j in J
problem_4.r3 = ConstraintList()
for j in problem_4.J:
    problem_4.r3.add(sum(problem_4.x[i,j] for i in problem_4.I) <= problem_4.M * problem_4.y[j])

# sum{i in I} x[i,j] >= y[j] for all j in J
problem_4.r4 = ConstraintList()
for j in problem_4.J:
    problem_4.r4.add(sum(problem_4.x[i,j] for i in problem_4.I) >= problem_4.y[j])

# sum{j in J} y[j] >= 2
problem_4.r5 = Constraint(expr=sum(problem_4.y[j] for j in problem_4.J) >= 2)


# at least 5 of i must be sent to each j
problem_4.r6 = ConstraintList()
for j in problem_4.J:
    problem_4.r6.add(sum(problem_4.x[i,j] for i in problem_4.I) >= 5)

# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_4)

# Imprimiendo resultados
print('Objective function value: ', problem_4.z())
for i in problem_4.I:
    for j in problem_4.J:
        print('x[',i,',',j,'] = ', problem_4.x[i,j]())





Objective function value:  -18.985730624258565
x[ 1 , 1 ] =  -0.0
x[ 1 , 2 ] =  4.0
x[ 1 , 3 ] =  -0.0
x[ 1 , 4 ] =  40.0
x[ 2 , 1 ] =  5.0
x[ 2 , 2 ] =  1.0
x[ 2 , 3 ] =  5.0
x[ 2 , 4 ] =  24.0


# Problema 5

In [14]:
problem_5 = ConcreteModel()

# set A 1..6
problem_5.A = RangeSet(1, 6)

# set T 1..6
problem_5.T = RangeSet(1, 6)

# set J 1..8
problem_5.J = RangeSet(1, 8)

# set AA :={1..6 by 2}
problem_5.AA = Set(initialize=[i for i in problem_5.A if i % 2 == 1])

demand = {(1,1): 34, (1,2): 246, (1,3): 40, (1,4): 375, (1,5): 81, (1,6): 255,
          (2,1): 87, (2,2): 107, (2,3): 57, (2,4): 157, (2,5): 44, (2,6): 398,
          (3,1): 52, (3,2): 128, (3,3): 87, (3,4): 400, (3,5): 40, (3,6): 469,
          (4,1): 32, (4,2): 406, (4,3): 39, (4,4): 333, (4,5): 34, (4,6): 418,
          (5,1): 92, (5,2): 144, (5,3): 37, (5,4): 207, (5,5): 31, (5,6): 440,
          (6,1): 33, (6,2): 284, (6,3): 73, (6,4): 448, (6,5): 57, (6,6): 182}

# param Dem{A,T}
problem_5.Dem = Param(problem_5.A, problem_5.T, initialize=demand)

# param Cap
problem_5.Cap = Param(initialize=8*10)

timeontask = {(1,1): 1, (1,2): 1, (1,3): 2, (1,4): 3, (1,5): 2, (1,6): 3, (1,7): 1, (1,8): 1,
              (2,1): 1, (2,2): 1, (2,3): 1, (2,4): 0, (2,5): 2, (2,6): 0, (2,7): 1, (2,8): 2,
              (3,1): 1, (3,2): 1, (3,3): 2, (3,4): 3, (3,5): 1, (3,6): 1, (3,7): 3, (3,8): 0,
              (4,1): 3, (4,2): 2, (4,3): 2, (4,4): 0, (4,5): 1, (4,6): 3, (4,7): 3, (4,8): 0,
              (5,1): 0, (5,2): 0, (5,3): 1, (5,4): 2, (5,5): 0, (5,6): 1, (5,7): 0, (5,8): 1,
              (6,1): 1, (6,2): 0, (6,3): 3, (6,4): 3, (6,5): 2, (6,6): 2, (6,7): 3, (6,8): 3}

# param a{A,J}
problem_5.a = Param(problem_5.A, problem_5.J, initialize=timeontask)

inv_cost = {1: 1, 2: 2, 3: 1, 4: 1, 5: 6, 6: 3}

# param h{A}
problem_5.h = Param(problem_5.A, initialize=inv_cost)

utility = {1: 27, 2: 38, 3: 36, 4: 28, 5: 19, 6: 39}

# param r{A}
problem_5.r = Param(problem_5.A, initialize=utility)

workdays = {1: 27, 2: 24, 3: 23, 4: 24, 5: 26, 6: 20}

# param Dia{T}
problem_5.Dia = Param(problem_5.T, initialize=workdays)

# var x{A,T} >= 0 integer
problem_5.x = Var(problem_5.A, problem_5.T, within=NonNegativeIntegers)

# var I{A, {0} union T} >= 0 integer
problem_5.I = Var(problem_5.A,list({0}.union(list(problem_5.T))), within=NonNegativeIntegers)

# var s{A,T} >= 0 integer
problem_5.s = Var(problem_5.A, problem_5.T, within=NonNegativeIntegers)

# var y binary
problem_5.y = Var(within=Binary)

# maximize z sum{i in A, t in T} (s[i,t]*r[i] - I[i,t]*h[i])
problem_5.z = Objective(expr=sum(problem_5.s[i,t]*problem_5.r[i] - problem_5.I[i,t]*problem_5.h[i] for i in problem_5.A for t in problem_5.T), sense=maximize)

# define constraints
# s[i,t] <= Dem[i,t] forall i in A, t in T
problem_5.r1 = ConstraintList()
for i in problem_5.A:
    for t in problem_5.T:
        problem_5.r1.add(problem_5.s[i,t] <= problem_5.Dem[i,t])

# sum{j in J, i in AA} x[i,t] * a[i,j] <= Cap * Dia[t] * y forall t in T
problem_5.r2 = ConstraintList()
for t in problem_5.T:
    problem_5.r2.add(sum(problem_5.x[i,t] * problem_5.a[i,j] for j in problem_5.J for i in problem_5.AA) <= problem_5.Cap * problem_5.Dia[t] * problem_5.y)

# sum{j in J, i in A diff AA} x[i,t] * a[i,j] <= Cap * Dia[t] * (1-y) forall t in T
problem_5.r3 = ConstraintList()
for t in problem_5.T:
    problem_5.r3.add(sum(problem_5.x[i,t] * problem_5.a[i,j] for j in problem_5.J for i in problem_5.A.difference(problem_5.AA)) <= problem_5.Cap * problem_5.Dia[t] * (1-problem_5.y))

# x[i,t] >= 10 * y for t in T, i in AA
problem_5.r4 = ConstraintList()
for t in problem_5.T:
    for i in problem_5.AA:
        problem_5.r4.add(problem_5.x[i,t] >= 10 * problem_5.y)

# x[i,t] >= 10 * (1-y) for t in T, i in A diff AA
problem_5.r5 = ConstraintList()
for t in problem_5.T:
    for i in problem_5.A.difference(problem_5.AA):
        problem_5.r5.add(problem_5.x[i,t] >= 10 * (1-problem_5.y))

# I[i,t-1] + x[i,t] = I[i,t] + s[i,t] forall t in T, i in A
problem_5.r6 = ConstraintList()
for t in problem_5.T:
    for i in problem_5.A:
        problem_5.r6.add(problem_5.I[i,t-1] + problem_5.x[i,t] == problem_5.I[i,t] + problem_5.s[i,t])

# I[i,0] = 0 forall i in A
problem_5.r7 = ConstraintList()
for i in problem_5.A:
    problem_5.r7.add(problem_5.I[i,0] == 0)

# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_5)

# Imprimiendo resultados
print('Objective function value: ', problem_5.z())
for i in problem_5.A:
    for t in problem_5.T:
        print('x[', i, ',', t, ']: ', problem_5.x[i,t]())




Objective function value:  42092.0
y:  0.0
x[ 1 , 1 ]:  0.0
x[ 1 , 2 ]:  0.0
x[ 1 , 3 ]:  0.0
x[ 1 , 4 ]:  0.0
x[ 1 , 5 ]:  0.0
x[ 1 , 6 ]:  0.0
x[ 2 , 1 ]:  91.0
x[ 2 , 2 ]:  106.0
x[ 2 , 3 ]:  70.0
x[ 2 , 4 ]:  201.0
x[ 2 , 5 ]:  221.0
x[ 2 , 6 ]:  161.0
x[ 3 , 1 ]:  0.0
x[ 3 , 2 ]:  0.0
x[ 3 , 3 ]:  0.0
x[ 3 , 4 ]:  0.0
x[ 3 , 5 ]:  0.0
x[ 3 , 6 ]:  0.0
x[ 4 , 1 ]:  10.0
x[ 4 , 2 ]:  11.0
x[ 4 , 3 ]:  10.0
x[ 4 , 4 ]:  10.0
x[ 4 , 5 ]:  10.0
x[ 4 , 6 ]:  10.0
x[ 5 , 1 ]:  0.0
x[ 5 , 2 ]:  0.0
x[ 5 , 3 ]:  0.0
x[ 5 , 4 ]:  0.0
x[ 5 , 5 ]:  0.0
x[ 5 , 6 ]:  0.0
x[ 6 , 1 ]:  76.0
x[ 6 , 2 ]:  54.0
x[ 6 , 3 ]:  67.0
x[ 6 , 4 ]:  10.0
x[ 6 , 5 ]:  10.0
x[ 6 , 6 ]:  10.0


In [17]:
problem_6 = ConcreteModel()

# define sets I 1..3
problem_6.I = RangeSet(1,3)

# define sets J 1..3
problem_6.J = RangeSet(1,3)

cost = {1: 380000, 2: 195000, 3: 47000}

# param C{J}
problem_6.C = Param(problem_6.J, initialize=cost)

sale_price = {1: 460000, 2: 260000, 3: 70000}

# param V{J}
problem_6.V = Param(problem_6.J, initialize=sale_price)

prod_weight = {1: 40, 2: 28, 3: 10}

# param PE{J}
problem_6.PE = Param(problem_6.J, initialize=prod_weight)

budget = {1: 1000000, 2: 5000000, 3: 0}

# param PRES{I}
problem_6.PRESU = Param(problem_6.I, initialize=budget)

# param M
problem_6.M = Param(initialize=10000000)

# var x{I,J} >= 0 integer
problem_6.x = Var(problem_6.I, problem_6.J, within=NonNegativeIntegers)

# var y{I} binary
problem_6.y = Var(problem_6.I, within=Binary)

# var A{I} >= 0
problem_6.A = Var(problem_6.I, within=NonNegativeReals)

# var SP{I} >= 0
problem_6.SP = Var(problem_6.I, within=NonNegativeReals)

# maximize z sum {i in I, j in J} x[i,j] * (V[j] - C[j]) - (sum {i in I} 2000 * SP[i] + sum {i in I} A[i])
problem_6.z = Objective(expr=sum(problem_6.x[i,j] * (problem_6.V[j] - problem_6.C[j]) for i in problem_6.I for j in problem_6.J) - (sum(2000 * problem_6.SP[i] for i in problem_6.I) + sum(problem_6.A[i] for i in problem_6.I)), sense=maximize)

# subject to
# sum {j in J} x[i,j] * PE[j] <= 50 + SP[i] forall i in I
problem_6.r1 = ConstraintList()
for i in problem_6.I:
    problem_6.r1.add(sum(problem_6.x[i,j] * problem_6.PE[j] for j in problem_6.J) <= 50 + problem_6.SP[i])

# sum {j in J} x[i,j] * C[j] <= 3000000 * y[i] * PRESU[i] forall i in I
problem_6.r2 = ConstraintList()
for i in problem_6.I:
    problem_6.r2.add(sum(problem_6.x[i,j] * problem_6.C[j] for j in problem_6.J) <= 3000000 + problem_6.y[i] * problem_6.PRESU[i])

# A[i] >= sum{j in I} x[i,j] * C[j] * 0.15 - (1-y[i]) * M forall i in I
problem_6.r3 = ConstraintList()
for i in problem_6.I:
    problem_6.r3.add(problem_6.A[i] >= sum(problem_6.x[i,j] * problem_6.C[j] * 0.15 for j in problem_6.J) - (1-problem_6.y[i]) * problem_6.M)

# solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_6)

print('Solución optima:')
print('Valor óptimo:', value(problem_6.z))
for x in problem_6.x:
    print(x, value(problem_6.x[x]))
print('Valor óptimo:', value(problem_6.z))

Solución optima:
Valor óptimo: 867000.0
(1, 1) -0.0
(1, 2) -0.0
(1, 3) 63.0
(2, 1) -0.0
(2, 2) -0.0
(2, 3) 63.0
(3, 1) -0.0
(3, 2) -0.0
(3, 3) 63.0
Valor óptimo: 867000.0


In [19]:
problem_7 = ConcreteModel()

# param m
problem_7.m = Param(initialize=5)

# param n
problem_7.n = Param(initialize=6)

# param M
problem_7.M = Param(initialize=10000)

# set I 1..m
problem_7.I = RangeSet(1, problem_7.m)

# set J 1..n
problem_7.J = RangeSet(1, problem_7.n)

timeontask = {(1,1): 6,  (1,2): 1, (1,3): 10, (1,4): 2, (1,5): 4,  (1,6): 2,
              (2,1): 6,  (2,2): 1, (2,3): 2,  (2,4): 6, (2,5): 9,  (2,6): 6,
              (3,1): 1,  (3,2): 4, (3,3): 3,  (3,4): 3, (3,5): 8,  (3,6): 4,
              (4,1): 10, (4,2): 4, (4,3): 10, (4,4): 8, (4,5): 6,  (4,6): 4,
              (5,1): 1,  (5,2): 2, (5,3): 9,  (5,4): 6, (5,5): 10, (5,6): 5}

# param P{I,J}
problem_7.P = Param(problem_7.I, problem_7.J, initialize=timeontask)

spec_order = {(1,1): 1, (1,2): 2, (1,3): 3, (1,4): 3, (1,5): 4, (1,6): 5,
              (2,1): 3, (2,2): 3, (2,3): 1, (2,4): 2, (2,5): 3, (2,6): 1,
              (3,1): 5, (3,2): 5, (3,3): 5, (3,4): 4, (3,5): 1, (3,6): 4,
              (4,1): 2, (4,2): 4, (4,3): 4, (4,4): 1, (4,5): 2, (4,6): 3,
              (5,1): 4, (5,2): 1, (5,3): 2, (5,4): 5, (5,5): 5, (5,6): 2}

# param theta{I,J}
problem_7.theta = Param(problem_7.I, problem_7.J, initialize=spec_order)

# var x{I,J} >= 0
problem_7.x = Var(problem_7.I, problem_7.J, within=NonNegativeReals)

# var z{I,J,J} binary
problem_7.z = Var(problem_7.I, problem_7.J, problem_7.J, within=Binary)

# var Cmax >= 0
problem_7.Cmax = Var(within=NonNegativeReals)

# minimize makespan: Cmax
problem_7.makespan = Objective(expr=problem_7.Cmax, sense=minimize)

# subject to
# x[theta[h,j],j] >= x[theta[h-1,j],j] + P[theta[h-1,j],j] for all j in J, h in {2..m}}
problem_7.r1 = ConstraintList()
for j in problem_7.J:
    for h in range(2, problem_7.m+1):
        problem_7.r1.add(problem_7.x[problem_7.theta[h,j],j] >= problem_7.x[problem_7.theta[h-1,j],j] + problem_7.P[problem_7.theta[h-1,j],j])

# x[i,j] >= x[i,k] + P[i,k] - M * z[i,j,k] for all j in J, k in J, i in I: j < k
problem_7.r2 = ConstraintList()
for j in problem_7.J:
    for k in problem_7.J:
        if j < k:
            for i in problem_7.I:
                problem_7.r2.add(problem_7.x[i,j] >= problem_7.x[i,k] + problem_7.P[i,k] - problem_7.M * problem_7.z[i,j,k])

# x[i,k] >= x[i,j] + P[i,j] - M * (1 - z[i,j,k]) for all j in J, k in J, i in I: j < k
problem_7.r3 = ConstraintList()
for j in problem_7.J:
    for k in problem_7.J:
        if j < k:
            for i in problem_7.I:
                problem_7.r3.add(problem_7.x[i,k] >= problem_7.x[i,j] + problem_7.P[i,j] - problem_7.M * (1 - problem_7.z[i,j,k]))

# Cmax >= x[theta[m,j],j] + P[theta[m,j],j] for all j in J
problem_7.r4 = ConstraintList()
for j in problem_7.J:
    problem_7.r4.add(problem_7.Cmax >= problem_7.x[problem_7.theta[problem_7.m,j],j] + problem_7.P[problem_7.theta[problem_7.m,j],j])


solver = SolverFactory('gurobi')
results = solver.solve(problem_7)

print('Solución optima:')
for x in problem_7.x:
    print(x, value(problem_7.x[x]))
print('Valor óptimo:', value(problem_7.makespan))

Solución optima:
(1, 1) 0.0
(1, 2) 24.99999999998179
(1, 3) 5.999999999998916
(1, 4) 22.99999999998179
(1, 5) 18.99999999998179
(1, 6) 15.999999999999922
(2, 1) 9.0
(2, 2) 0.0
(2, 3) 42.99999999999892
(2, 4) 3.0
(2, 5) 22.99999999998179
(2, 6) 31.999999999989818
(3, 1) 6.0
(3, 2) 7.0
(3, 3) 2.9999999999989164
(3, 4) 0.0
(3, 5) 11.0
(3, 6) 27.999999999989818
(4, 1) 34.99999999999892
(4, 2) 17.0
(4, 3) 24.999999999998916
(4, 4) 9.0
(4, 5) 0.0
(4, 6) 20.999999999998916
(5, 1) 8.0
(5, 2) 13.999999999998916
(5, 3) 15.999999999998916
(5, 4) 28.99999999999892
(5, 5) 34.99999999999892
(5, 6) 0.0
Valor óptimo: 44.99999999999892


In [30]:
problem_8 = ConcreteModel()

# set I 1..3
problem_8.I = RangeSet(1, 3)

# set T 1..5
problem_8.T = RangeSet(1, 5)

exchange_rates = {(1,1,1): 0.95, (1,2,1): 0.00032, (1,3,1): 0.00028, 
                  (2,1,1): 3123, (2,2,1): 0.95, (2,3,1): 0.88,
                  (3,1,1): 3550, (3,2,1): 1.1363, (3,3,1): 0.95,
                  (1,1,2): 0.95, (1,2,2): 0.00032, (1,3,2): 0.00028,
                  (2,1,2): 3200, (2,2,2): 0.95, (2,3,2): 0.8,
                  (3,1,2): 3541, (3,2,2): 1.4, (3,3,2): 0.95,
                  (1,1,3): 0.95, (1,2,3): 0.00058, (1,3,3): 0.00045,
                  (2,1,3): 3123, (2,2,3): 0.95, (2,3,3): 0.9,
                  (3,1,3): 3541, (3,2,3): 1.14, (3,3,3): 0.95,
                  (1,1,4): 0.95, (1,2,4): 0.00032, (1,3,4): 0.00028,
                  (2,1,4): 3250, (2,2,4): 0.95, (2,3,4): 0.75,
                  (3,1,4): 3400, (3,2,4): 1.1363, (3,3,4): 0.95}

# param r{I,I,T} >= 0
problem_8.r = Param(problem_8.I, problem_8.I, problem_8.T, initialize=exchange_rates)

# var x{I,I,T} >= 0
problem_8.x = Var(problem_8.I, problem_8.I, problem_8.T, within=NonNegativeReals)

# maximize z sum{i in I} x[3,i,5]
problem_8.z = Objective(expr=sum(problem_8.x[3,i,5] for i in problem_8.I), sense=maximize)

# subject to
# sum{i in I} x[1,i,1] <= 100000
problem_8.r1 = Constraint(expr=sum(problem_8.x[1,i,1] for i in problem_8.I) <= 100000)

# sum{i in I} x[j,i,1] = 0 for j in {2,3}
problem_8.r2 = ConstraintList()
for j in {2,3}:
    problem_8.r2.add(sum(problem_8.x[j,i,1] for i in problem_8.I) == 0)

# sum{m in I} x[m,j,t] * r[m,j,t] = sum{m in I} x[j,m,t+1] for j in I, t in T diff {5}
problem_8.r3 = ConstraintList()
for j in problem_8.I:
    for t in problem_8.T:
        if t < 5:
            problem_8.r3.add(sum(problem_8.x[m,j,t] * problem_8.r[m,j,t] for m in problem_8.I) == sum(problem_8.x[j,m,t+1] for m in problem_8.I))

solver = SolverFactory('gurobi')
results = solver.solve(problem_8)

print('Solución optima:')
for i in problem_8.I:
    for j in problem_8.I:
        for t in problem_8.T:
            print('x[',i,',',j,',',t,'] = ', value(problem_8.x[i,j,t]))
print('Valor óptimo:', value(problem_8.z))

Solución optima:


TypeError: 'NoneType' object is not callable

# propuesta estudiante 

In [35]:
problem_9 = ConcreteModel()

# Define the sets
problem_9.I = RangeSet(1, 6)  # Muelle
problem_9.J = RangeSet(1, 10)  # Productos
problem_9.A = Set(initialize=[(1, 2), (1, 9), (2, 8), (8, 9)])

cost = {
    (1, 1): 6.84,   (1, 2): 281.12, (1, 3): 141.51, (1, 4): 317.79, (1, 5): 390.45, (1, 6): 231.25, (1, 7): 7.73,   (1, 8): 34.34,  (1, 9): 32.76,  (1, 10): 156.86,
    (2, 1): 343.14, (2, 2): 168.57, (2, 3): 0.99,   (2, 4): 107.09, (2, 5): 87.1,   (2, 6): 112.49, (2, 7): 211,    (2, 8): 141.58, (2, 9): 91.99,  (2, 10): 257.41,
    (3, 1): 346.04, (3, 2): 274.48, (3, 3): 196.41, (3, 4): 279.88, (3, 5): 375.14, (3, 6): 114.51, (3, 7): 16.41,  (3, 8): 346.89, (3, 9): 472.78, (3, 10): 430.69,
    (4, 1): 286.73, (4, 2): 484.27, (4, 3): 397.35, (4, 4): 273.22, (4, 5): 385.99, (4, 6): 53.26,  (4, 7): 458.01, (4, 8): 472.33, (4, 9): 172.28, (4, 10): 338.64,
    (5, 1): 244.52, (5, 2): 334.76, (5, 3): 495.84, (5, 4): 395.05, (5, 5): 160.09, (5, 6): 420.11, (5, 7): 166.22, (5, 8): 88.61,  (5, 9): 1.19,   (5, 10): 257.7,
    (6, 1): 284.39, (6, 2): 440.89, (6, 3): 281.01, (6, 4): 164.32, (6, 5): 39.73,  (6, 6): 225.61, (6, 7): 470.88, (6, 8): 152.78, (6, 9): 368.29, (6, 10): 276.47,
}

capacity = {1: 40, 2: 40, 3: 60, 4: 30, 5: 60, 6: 60, 7: 60, 8: 60, 9: 40, 10: 50}

demand = {
    1: 60,
    2: 80,
    3: 120,
    4: 45,
    5: 120,
    6: 90,
    7: 90,
    8: 120,
    9: 60,
    10: 75
}

# Define the parameters
problem_9.Cap = Param(problem_9.J, initialize=capacity)  # Cantidad de producto j que se puede almacenar en cada muelle
problem_9.Dem = Param(problem_9.J, initialize=demand)  # Cantidad requerida de producto j en bodega
problem_9.C = Param(problem_9.I, problem_9.J, initialize=cost)  # Costo manejo y mantenimiento de producto j en el muelle i
problem_9.M = Param(initialize=100000) 


# Define the variables
problem_9.X = Var(problem_9.I, problem_9.J, within=NonNegativeReals)  # Cantidad de producto j a almacenar en muelle i
problem_9.Y = Var(problem_9.I, problem_9.J, within=Binary)  # 1: Si el producto j se almacena en la muelle i, 0: E.O.C
problem_9.CONmas = Var(problem_9.I, within=NonNegativeReals)  # Esta variable funciona como contador para la restricción 7
problem_9.CONmen = Var(problem_9.I, within=NonNegativeReals)  # Esta variable funciona como contador para la restricción 7
problem_9.CostoReal = Var(within=NonNegativeReals)

# Define the objective function
problem_9.CostoTotal = Objective(
    expr=problem_9.CostoReal + sum(problem_9.CONmas[i] + problem_9.CONmen[i] for i in problem_9.I),
    sense=minimize,
)

# Define the constraints
problem_9.r1 = ConstraintList()
for i in problem_9.I:
    for j in problem_9.J:
        problem_9.r1.add(problem_9.X[i, j] <= problem_9.Cap[j])

problem_9.r2 = ConstraintList()
for j in problem_9.J:
    problem_9.r2.add(sum(problem_9.X[i, j] for i in problem_9.I) >= problem_9.Dem[j])

problem_9.r3 = ConstraintList()
for i in problem_9.I:
    for j in problem_9.J:
        problem_9.r3.add(problem_9.X[i, j] <= problem_9.Y[i, j] * problem_9.M)

problem_9.r4 = ConstraintList()
for i in problem_9.I:
    for j in problem_9.J:
        problem_9.r4.add(problem_9.X[i, j] >= problem_9.Y[i, j])

problem_9.r5 = ConstraintList()
for i in problem_9.I:
    for k, m in problem_9.A:
        problem_9.r5.add(problem_9.Y[i, k] <= 1 - problem_9.Y[i, m])

problem_9.r6 = ConstraintList()
for i in problem_9.I:
    problem_9.r6.add(problem_9.X[i, 5] == problem_9.X[i, 6])

problem_9.r7 = ConstraintList()
for i in problem_9.I:
    problem_9.r7.add(problem_9.X[i, 3] - problem_9.X[i, 4] == problem_9.CONmas[i] - problem_9.CONmen[i])

problem_9.r8 = Constraint(expr=problem_9.CostoReal == sum(problem_9.C[i, j] * problem_9.X[i, j] for i in problem_9.I for j in problem_9.J))

solver = SolverFactory('gurobi')
results = solver.solve(problem_9)

# Print the output
print(f"Status = {results.solver.status}")
print(f"Termination condition = {results.solver.termination_condition}")
print(f"Objective = {problem_9.CostoTotal()}")
for i in problem_9.I:
    for j in problem_9.J:
        print(f"X[{i},{j}] = {problem_9.X[i, j].value}")



Status = ok
Termination condition = optimal
Objective = 94260.65000000001
X[1,1] = 40.0
X[1,2] = -0.0
X[1,3] = 60.0
X[1,4] = 0.0
X[1,5] = 0.0
X[1,6] = 0.0
X[1,7] = 60.0
X[1,8] = 60.0
X[1,9] = 0.0
X[1,10] = 50.0
X[2,1] = -0.0
X[2,2] = 40.0
X[2,3] = 60.0
X[2,4] = 30.0
X[2,5] = 60.0
X[2,6] = 60.0
X[2,7] = 0.0
X[2,8] = 0.0
X[2,9] = 20.0
X[2,10] = 25.0
X[3,1] = -0.0
X[3,2] = 40.0
X[3,3] = 0.0
X[3,4] = 0.0
X[3,5] = 0.0
X[3,6] = 0.0
X[3,7] = 30.0
X[3,8] = -0.0
X[3,9] = -0.0
X[3,10] = 0.0
X[4,1] = -0.0
X[4,2] = -0.0
X[4,3] = 0.0
X[4,4] = 0.0
X[4,5] = 0.0
X[4,6] = 0.0
X[4,7] = 0.0
X[4,8] = -0.0
X[4,9] = -0.0
X[4,10] = 0.0
X[5,1] = 0.0
X[5,2] = -0.0
X[5,3] = 0.0
X[5,4] = 0.0
X[5,5] = 0.0
X[5,6] = 0.0
X[5,7] = 0.0
X[5,8] = 0.0
X[5,9] = 40.0
X[5,10] = 0.0
X[6,1] = 20.0
X[6,2] = -0.0
X[6,3] = 0.0
X[6,4] = 15.0
X[6,5] = 60.0
X[6,6] = 60.0
X[6,7] = 0.0
X[6,8] = 60.0
X[6,9] = -0.0
X[6,10] = 0.0


In [37]:
problem_10 = ConcreteModel()

# Define the sets
problem_10.T = RangeSet(1, 5)  # Set T
problem_10.P = RangeSet(0, 8)  # Set P
problem_10.SUC = Set(within=problem_10.P * problem_10.P, initialize=[(0, 2), (0, 3), (1, 3), (1, 4), (2, 5), (2, 6), (3, 6), (3, 7), (4, 7), (4, 8)])

cost = {
    (0, 1): 7, (0, 2): 10, (0, 3): 8, (0, 4): 11, (0, 5): 13,
    (1, 1): 11, (1, 2): 9, (1, 3): 10, (1, 4): 13, (1, 5): 10,
    (2, 1): 9, (2, 2): 7, (2, 3): 10, (2, 4): 12, (2, 5): 8,
    (3, 1): 12, (3, 2): 9, (3, 3): 12, (3, 4): 9, (3, 5): 8,
    (4, 1): 9, (4, 2): 13, (4, 3): 7, (4, 4): 11, (4, 5): 10,
    (5, 1): 11, (5, 2): 7, (5, 3): 12, (5, 4): 13, (5, 5): 8,
    (6, 1): 9, (6, 2): 13, (6, 3): 8, (6, 4): 11, (6, 5): 9,
    (7, 1): 8, (7, 2): 10, (7, 3): 12, (7, 4): 9, (7, 5): 10,
    (8, 1): 10, (8, 2): 12, (8, 3): 13, (8, 4): 8, (8, 5): 11,
}

# Define the parameter
problem_10.C = Param(problem_10.P, problem_10.T, initialize = cost)

# Define the variables
problem_10.x = Var(problem_10.P, problem_10.T, within=Binary)

# Define the objective function
problem_10.z = Objective(expr=sum(problem_10.C[p, t] * problem_10.x[p, t] for p in problem_10.P for t in problem_10.T), sense=maximize)

# Define the constraints
problem_10.r = ConstraintList()
for p in problem_10.P:
    problem_10.r.add(sum(problem_10.x[p, t] for t in problem_10.T) == 1)

problem_10.Suce = ConstraintList()
for i, j in problem_10.SUC:
    problem_10.Suce.add(sum(t * problem_10.x[j, t] for t in problem_10.T) <= sum(t * problem_10.x[i, t] for t in problem_10.T))

solver = SolverFactory('gurobi')
results = solver.solve(problem_10)

# Print the output
print(f"Status = {results.solver.status}")
print(f"Termination condition = {results.solver.termination_condition}")
print(f"Objective = {problem_10.z()}")
for x in problem_10.x:
    print(f"x[{x}] = {problem_10.x[x].value}")

Status = ok
Termination condition = optimal
Objective = 112.0
x[(0, 1)] = 0.0
x[(0, 2)] = 0.0
x[(0, 3)] = 0.0
x[(0, 4)] = 0.0
x[(0, 5)] = 1.0
x[(1, 1)] = 0.0
x[(1, 2)] = 0.0
x[(1, 3)] = 0.0
x[(1, 4)] = 1.0
x[(1, 5)] = 0.0
x[(2, 1)] = 0.0
x[(2, 2)] = 0.0
x[(2, 3)] = 0.0
x[(2, 4)] = 1.0
x[(2, 5)] = 0.0
x[(3, 1)] = 0.0
x[(3, 2)] = 0.0
x[(3, 3)] = 1.0
x[(3, 4)] = 0.0
x[(3, 5)] = 0.0
x[(4, 1)] = 0.0
x[(4, 2)] = 0.0
x[(4, 3)] = 0.0
x[(4, 4)] = 1.0
x[(4, 5)] = 0.0
x[(5, 1)] = 0.0
x[(5, 2)] = 0.0
x[(5, 3)] = 0.0
x[(5, 4)] = 1.0
x[(5, 5)] = 0.0
x[(6, 1)] = 0.0
x[(6, 2)] = 1.0
x[(6, 3)] = 0.0
x[(6, 4)] = 0.0
x[(6, 5)] = 0.0
x[(7, 1)] = 0.0
x[(7, 2)] = 0.0
x[(7, 3)] = 1.0
x[(7, 4)] = 0.0
x[(7, 5)] = 0.0
x[(8, 1)] = 0.0
x[(8, 2)] = 0.0
x[(8, 3)] = 1.0
x[(8, 4)] = 0.0
x[(8, 5)] = 0.0


# Propuesta Estudiante

In [39]:
problem_10 = ConcreteModel()

# Define the sets
problem_10.T = RangeSet(1, 5)  # Set T
problem_10.P = RangeSet(0, 8)  # Set P
problem_10.SUC = Set(within=problem_10.P * problem_10.P, initialize=[(0, 2), (0, 3), (1, 3), (1, 4), (2, 5), (2, 6), (3, 6), (3, 7), (4, 7), (4, 8)])

cost = {
    (0, 1): 7, (0, 2): 10, (0, 3): 8, (0, 4): 11, (0, 5): 13,
    (1, 1): 11, (1, 2): 9, (1, 3): 10, (1, 4): 13, (1, 5): 10,
    (2, 1): 9, (2, 2): 7, (2, 3): 10, (2, 4): 12, (2, 5): 8,
    (3, 1): 12, (3, 2): 9, (3, 3): 12, (3, 4): 9, (3, 5): 8,
    (4, 1): 9, (4, 2): 13, (4, 3): 7, (4, 4): 11, (4, 5): 10,
    (5, 1): 11, (5, 2): 7, (5, 3): 12, (5, 4): 13, (5, 5): 8,
    (6, 1): 9, (6, 2): 13, (6, 3): 8, (6, 4): 11, (6, 5): 9,
    (7, 1): 8, (7, 2): 10, (7, 3): 12, (7, 4): 9, (7, 5): 10,
    (8, 1): 10, (8, 2): 12, (8, 3): 13, (8, 4): 8, (8, 5): 11,
}

# Define the parameter
problem_10.C = Param(problem_10.P, problem_10.T, initialize = cost)

# Define the variables
problem_10.x = Var(problem_10.P, problem_10.T, within=Binary)

# Define the objective function
problem_10.z = Objective(expr=sum(problem_10.C[p, t] * problem_10.x[p, t] for p in problem_10.P for t in problem_10.T), sense=maximize)

# Define the constraints
problem_10.r = ConstraintList()
for p in problem_10.P:
    problem_10.r.add(sum(problem_10.x[p, t] for t in problem_10.T) == 1)

problem_10.Suce = ConstraintList()
for i, j in problem_10.SUC:
    problem_10.Suce.add(sum(t * problem_10.x[j, t] for t in problem_10.T) <= sum(t * problem_10.x[i, t] for t in problem_10.T))

# add constraint that limits extraction to 7 mines total


solver = SolverFactory('gurobi')
results = solver.solve(problem_10)

# Print the output
print(f"Status = {results.solver.status}")
print(f"Termination condition = {results.solver.termination_condition}")
print(f"Objective = {problem_10.z()}")
for x in problem_10.x:
    print(f"x[{x}] = {problem_10.x[x].value}")

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
Status = warning
Termination condition = infeasible
ERROR: evaluating object as numeric value: x[0,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[0,1]


ValueError: No value for uninitialized NumericValue object x[0,1]

In [43]:
problem_11 = ConcreteModel()

# Define the set
problem_11.I = RangeSet(1, 5)

cost = {
    (1, 1): None, (1, 2): 5, (1, 3): 3, (1, 4): 2, (1, 5): 1,
    (2, 1): 2, (2, 2): None, (2, 3): 4, (2, 4): 1, (2, 5): 2,
    (3, 1): 3, (3, 2): 5, (3, 3): None, (3, 4): 2, (3, 5): 2,
    (4, 1): 1, (4, 2): 3, (4, 3): 4, (4, 4): None, (4, 5): 1,
    (5, 1): 1, (5, 2): 2, (5, 3): 2, (5, 4): 2, (5, 5): None,
}

demand = {
    1: 0,
    2: 10,
    3: 15,
    4: 5,
    5: 20,
}

# Define the parameters
problem_11.c = Param(problem_11.I, problem_11.I, initialize=cost)
problem_11.d = Param(problem_11.I, initialize=demand)
problem_11.M = Param(initialize=9000)

# Define the variables
problem_11.x = Var(problem_11.I, problem_11.I, within=Binary, initialize=0)
problem_11.y = Var(problem_11.I, problem_11.I, within=NonNegativeReals, initialize=0)

# Define the objective function
problem_11.z = Objective(expr=sum(problem_11.y[i, j] * problem_11.c[i, j] for i in problem_11.I for j in problem_11.I if i != j))

# Define the constraints
problem_11.rsalid = ConstraintList()
for i in problem_11.I:
    problem_11.rsalid.add(sum(problem_11.x[i, j] for j in problem_11.I if j != i) == 1)

problem_11.rllega = ConstraintList()
for j in problem_11.I:
    problem_11.rllega.add(sum(problem_11.x[i, j] for i in problem_11.I if i != j) == 1)

problem_11.rela = ConstraintList()
for i in problem_11.I:
    for j in problem_11.I:
        if i != j:
            problem_11.rela.add(problem_11.y[i, j] <= problem_11.M * problem_11.x[i, j])

problem_11.rInv = ConstraintList()
for j in problem_11.I:
    if j != 1:
        problem_11.rInv.add(sum(problem_11.y[i, j] for i in problem_11.I if i != j) == problem_11.d[j] + sum(problem_11.y[j, i] for i in problem_11.I if i != j))

problem_11.rini = Constraint(expr=sum(problem_11.y[1, i] for i in problem_11.I if i != 1) == 50)

solver = SolverFactory('gurobi')
results = solver.solve(problem_11)

# Print the output
print(f"Status = {results.solver.status}")
print(f"Termination condition = {results.solver.termination_condition}")
print(f"Objective = {problem_11.z()}")
for x in problem_11.x:
    if problem_11.x[x].value > 0:
        print(f"x[{x}] = {problem_11.x[x].value}")


Status = ok
Termination condition = optimal
Objective = 170.0
x[(1, 5)] = 1.0
x[(2, 1)] = 1.0
x[(3, 4)] = 1.0
x[(4, 2)] = 1.0
x[(5, 3)] = 1.0


In [50]:
problem_12 = ConcreteModel()

# Define the sets
problem_12.I = RangeSet(1, 10)
problem_12.J = RangeSet(1, 4)

cost = {
    1: 40,
    2: 62,
    3: 60,
    4: 42,
    5: 65,
    6: 44,
    7: 32,
    8: 55,
    9: 30,
    10: 54,
}

demand = {
    1: 2,
    2: 3,
    3: 2,
    4: 4,
}

# Define the parameters
problem_12.C = Param(problem_12.I, initialize=cost)
problem_12.DEM = Param(problem_12.J, initialize=demand)
problem_12.M = Param(initialize=10000)

# Define the variables
problem_12.x = Var(problem_12.I, problem_12.J, within=Binary)
problem_12.P = Var(problem_12.J, within=NonNegativeReals)
problem_12.yma = Var(problem_12.I, problem_12.J, within=NonNegativeReals)
problem_12.yme = Var(problem_12.I, problem_12.J, within=NonNegativeReals)
problem_12.aux = Var(problem_12.I, problem_12.J)

# Define the objective function
problem_12.z = Objective(expr=sum(problem_12.yma[i, j] + problem_12.yme[i, j] for i in problem_12.I for j in problem_12.J))

# Define the constraints
problem_12.Dem = ConstraintList()
for j in problem_12.J:
    problem_12.Dem.add(sum(problem_12.x[i, j] for i in problem_12.I) >= problem_12.DEM[j])

problem_12.costos = ConstraintList()
for i in problem_12.I:
    for j in problem_12.J:
        problem_12.costos.add(problem_12.aux[i, j] == problem_12.yma[i, j] - problem_12.yme[i, j])

problem_12.aux1 = ConstraintList()
for i in problem_12.I:
    for j in problem_12.J:
        problem_12.aux1.add(problem_12.aux[i, j] <= problem_12.C[i] - problem_12.P[j] + problem_12.M * (1 - problem_12.x[i, j]))

problem_12.aux2 = ConstraintList()
for i in problem_12.I:
    for j in problem_12.J:
        problem_12.aux2.add(problem_12.aux[i, j] >= problem_12.C[i] - problem_12.P[j] - problem_12.M * (1 - problem_12.x[i, j]))

problem_12.salarioTurno = ConstraintList()
for i in problem_12.I:
    for j in problem_12.J:
        problem_12.salarioTurno.add(problem_12.P[j] <= problem_12.C[i] * problem_12.x[i, j] + problem_12.M * (1 - problem_12.x[i, j]))

problem_12.CosT = Constraint(expr=sum(problem_12.DEM[j] * problem_12.P[j] for j in problem_12.J) <= 600)

# Solve the problem
solver = SolverFactory('gurobi')
results = solver.solve(problem_12)

# Print the output
print(f"Status = {results.solver.status}")
print(f"Termination condition = {results.solver.termination_condition}")
print(f"Objective = {problem_12.z()}")
for x in problem_12.x:
    if problem_12.x[x].value > 0:
        print(f"x[{x}] = {problem_12.x[x].value}")


Status = ok
Termination condition = optimal
Objective = 23.0
x[(1, 2)] = 1.0
x[(2, 4)] = 1.0
x[(3, 4)] = 1.0
x[(4, 2)] = 1.0
x[(6, 2)] = 1.0
x[(8, 1)] = 1.0
x[(8, 3)] = 1.0
x[(8, 4)] = 1.0
x[(10, 1)] = 1.0
x[(10, 3)] = 1.0
x[(10, 4)] = 1.0


In [66]:
problem_12 = ConcreteModel()

# define set I 1..5
problem_12.I = RangeSet(1,5)

# define set J 1..4
problem_12.J = RangeSet(1,4)

# define set K 1..2
problem_12.K = RangeSet(1,2)

cost = {1: 50, 2: 30, 3: 35, 4: 60, 5: 40}

# Param C{I}
problem_12.C = Param(problem_12.I, initialize=cost)

demand = {(1,1): 3, (1,2): 3, (2,1): 2, (2,2): 2, (3,1): 2, (3,2): 2, (4,1): 4, (4,2): 3}

# Param DEM{J,K}
problem_12.DEM = Param(problem_12.J, problem_12.K, initialize=demand)

# Var Shifts{I}
problem_12.Shifts = Var(problem_12.I, within=NonNegativeIntegers)

problem_12.Target = Var(within=NonNegativeIntegers)

# var x{I,J,K} >= 0 integer
problem_12.x = Var(problem_12.I, problem_12.J, problem_12.K, within=NonNegativeIntegers)

# minimize sum {i in I, j in J, k in K} C[i] * x[i,j,k]
problem_12.z = Objective(expr=sum(problem_12.C[i] * problem_12.x[i,j,k] for i in problem_12.I for j in problem_12.J for k in problem_12.K), sense=minimize)

# define constraint sum {i in I} x[i,j,k] >= DEM[j] for all j in J and k in K
problem_12.r1 = ConstraintList()
for j in problem_12.J:
    for k in problem_12.K:
        problem_12.r1.add(sum(problem_12.x[i,j,k] for i in problem_12.I) >= problem_12.DEM[j,k])

# define constraint x[i,4,k] >= x[i,1,k] for all i in I and k in K
problem_12.r2 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r2.add(problem_12.x[i,4,k] >= problem_12.x[i,1,k])

# define constraint x[i,2,k] + x[i,3,k] <= 1 for all i in I and k in K
problem_12.r3 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r3.add(problem_12.x[i,2,k] + problem_12.x[i,3,k] <= 1)

# define constraint sum {j in {1..3}} x[i,j,k] <= 2 for all i in I and k in K
problem_12.r4 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r4.add(sum(problem_12.x[i,j,k] for j in range(1,4)) <= 2)

# define constraint sum {j in {2..4}} x[i,j,k] <= 2 for all i in I and k in K
problem_12.r5 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r5.add(sum(problem_12.x[i,j,k] for j in range(2,5)) <= 2)

# Constraint to enforce equality in the number of shifts for each nurse
problem_12.r6 = Constraint(expr=sum(problem_12.Shifts[i] for i in problem_12.I) == problem_12.Target)

# Constraint to ensure each nurse gets the same number of shifts
problem_12.r7 = ConstraintList()
for i in problem_12.I:
    problem_12.r7.add(problem_12.Shifts[i] == problem_12.Target)

solver = SolverFactory('gurobi')
results = solver.solve(problem_12)

print('Solución optima:')
for x in problem_12.x:
    print(x, value(problem_12.x[x]))
print('Valor óptimo:', value(problem_12.z))

Solución optima:
(1, 1, 1) -0.0
(1, 1, 2) -0.0
(1, 2, 1) 1.0
(1, 2, 2) -0.0
(1, 3, 1) 0.0
(1, 3, 2) 1.0
(1, 4, 1) 1.0
(1, 4, 2) -0.0
(2, 1, 1) 1.0
(2, 1, 2) 1.0
(2, 2, 1) 1.0
(2, 2, 2) -0.0
(2, 3, 1) -0.0
(2, 3, 2) 1.0
(2, 4, 1) 1.0
(2, 4, 2) 1.0
(3, 1, 1) 1.0
(3, 1, 2) 1.0
(3, 2, 1) 0.0
(3, 2, 2) 1.0
(3, 3, 1) 1.0
(3, 3, 2) 0.0
(3, 4, 1) 1.0
(3, 4, 2) 1.0
(4, 1, 1) -0.0
(4, 1, 2) -0.0
(4, 2, 1) -0.0
(4, 2, 2) -0.0
(4, 3, 1) -0.0
(4, 3, 2) -0.0
(4, 4, 1) -0.0
(4, 4, 2) -0.0
(5, 1, 1) 1.0
(5, 1, 2) 1.0
(5, 2, 1) 0.0
(5, 2, 2) 1.0
(5, 3, 1) 1.0
(5, 3, 2) 0.0
(5, 4, 1) 1.0
(5, 4, 2) 1.0
Valor óptimo: 780.0


In [53]:
problem_13 = ConcreteModel()

# Define the sets
problem_13.M = RangeSet(1, 4)
problem_13.I = RangeSet(1, 10)
problem_13.ES = problem_13.I - {1, 10}
problem_13.T = RangeSet(1, 6)
problem_13.A = Set(initialize=[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (2, 5), (2, 7),
                              (2, 9), (3, 2), (3, 7), (3, 8), (4, 2), (4, 5),
                              (5, 2), (5, 4), (5, 6), (5, 9), (6, 5), (6, 10), (7, 2), (7, 3),
                              (7, 8), (7, 9), (8, 3), (8, 7), (8, 9), (9, 2), (9, 5), (9, 7),
                              (9, 10), (9, 8)])
problem_13.INI = Set(initialize=[(1, 2), (1, 3), (1, 4)])
problem_13.FIN = Set(initialize=[(9, 10), (6, 10)])

maxper = {2: 5, 3: 8, 4: 10, 5: 5, 6: 14, 7: 10, 8: 12, 9: 15}
dis = {(1, 2): 4, (1, 3): 2, (1, 4): 5, (2, 1): 2, (2, 3): 3, (2, 4): 6, (2, 5): 7, (2, 7): 10, (2, 9): 10,
        (3, 1): 6, (3, 2): 3, (3, 7): 6, (3, 8): 9, (4, 1): 5, (4, 2): 7, (4, 5): 10, (5, 2): 8, (5, 4): 6,
        (5, 6): 9, (5, 9): 17, (6, 5): 3, (6, 10): 4, (7, 2): 3, (7, 3): 7, (7, 8): 10, (7, 9): 4, (8, 3): 15,
        (8, 7): 7, (8, 9): 13, (9, 2): 8, (9, 5): 4, (9, 7): 10, (9, 8): 13, (9, 10): 8, (10, 6): 3, (10, 9): 9}

# Define the parameters
problem_13.Dem = Param(initialize=30)
problem_13.MaxPer = Param(problem_13.ES, initialize=maxper)
problem_13.CapAv = Param(initialize=15)
problem_13.Dis = Param(problem_13.I, problem_13.I, initialize=dis)

# Define the variables
problem_13.x = Var(problem_13.M, problem_13.A, problem_13.T, within=Binary)
problem_13.y = Var(problem_13.M, problem_13.A, problem_13.T, within=NonNegativeIntegers)
problem_13.Days = Var(within=NonNegativeReals)

# Define the objective function
problem_13.obj = Objective(expr=sum(problem_13.x[m, i, j, t] * problem_13.Dis[i, j] +
                                    100 * problem_13.Days for m in problem_13.M
                                    for (i, j) in problem_13.A for t in problem_13.T))

# Define the constraints
problem_13.rdias = ConstraintList()
for m in problem_13.M:
    for (i, j) in problem_13.A:
        for t in problem_13.T:
            problem_13.rdias.add(problem_13.Days >= problem_13.x[m, i, j, t] * t)

problem_13.rMaxPer = ConstraintList()
for k in problem_13.ES:
    for t in problem_13.T:
        problem_13.rMaxPer.add(sum(problem_13.y[m, i, j, t] for (i, j) in problem_13.A for m in problem_13.M if j == k) <= problem_13.MaxPer[k])

problem_13.rIni = ConstraintList()
for m in problem_13.M:
    for (i, j) in problem_13.INI:
        for t in problem_13.T:
            problem_13.rIni.add(problem_13.y[m, i, j, t] == problem_13.Dem)

problem_13.rFIN = ConstraintList()
for m in problem_13.M:
    for (i, j) in problem_13.FIN:
        for t in problem_13.T:
            problem_13.rFIN.add(problem_13.y[m, i, j, t] == problem_13.Dem)

problem_13.rBalanceNodos = ConstraintList()
for k in problem_13.ES:
    for m in problem_13.M:
        for t in problem_13.T:
            problem_13.rBalanceNodos.add(sum(problem_13.y[m, i, j, t] for (i, j) in problem_13.A if j == k) ==
                                         sum(problem_13.y[m, j, i, t] for (j, i) in problem_13.A if j == k))

problem_13.rAviPeople1 = ConstraintList()
for m in problem_13.M:
    for (i, j) in problem_13.A:
        for t in problem_13.T:
            problem_13.rAviPeople1.add(problem_13.y[m, i, j, t] <= problem_13.x[m, i, j, t] * problem_13.CapAv)

problem_13.rAviPeople2 = ConstraintList()
for m in problem_13.M:
    for (i, j) in problem_13.A:
        for t in problem_13.T:
            problem_13.rAviPeople2.add(problem_13.y[m, i, j, t] >= problem_13.x[m, i, j, t])

problem_13.avionesNodo = ConstraintList()
for k in problem_13.I:
    for t in problem_13.T:
        for m in problem_13.M:
            problem_13.avionesNodo.add(sum(problem_13.x[m, i, j, t] for (i, j) in problem_13.A if i == k) <= 1)


# solve
solver = SolverFactory('gurobi')
solver.solve(problem_13)

# print results
print('Objective function value:')
print(problem_13.obj())




ValueError: Invalid constraint expression. The constraint expression resolved to a trivial Boolean (True) instead of a Pyomo object. Please modify your rule to return Constraint.Feasible instead of True.

Error thrown for Constraint 'avionesNodo[217]'

In [54]:
problem_14 = ConcreteModel()

# Define the sets
problem_14.I = RangeSet(1, 3)
problem_14.J = RangeSet(1, 3)
problem_14.K = RangeSet(1, 2)

demand = {1: 100, 2: 120, 3: 50}
costs = {
    (1, 1): 6, (1, 2): 7, (1, 3): 3,
    (2, 1): 5, (2, 2): 4, (2, 3): 5
}

amount = {1: 3, 2: 4, 3: 1.5}

# Define the parameters
problem_14.Dem = Param(problem_14.J, initialize=demand)
problem_14.Costos = Param(problem_14.K, problem_14.I, initialize=costs)
problem_14.CuanMad = Param(problem_14.J, initialize=amount)

# Define the variables
problem_14.x = Var(problem_14.I, problem_14.J, problem_14.K, within=NonNegativeIntegers)
problem_14.y = Var(problem_14.I, problem_14.K, within=Binary)

# Define the objective function
problem_14.obj = Objective(
    expr=sum(problem_14.Costos[k, i] * problem_14.x[i, j, k] for i in problem_14.I for j in problem_14.J for k in problem_14.K)
)

# Define the constraints
problem_14.dem = ConstraintList()
for j in problem_14.J:
    problem_14.dem.add(
        0.9 * sum(problem_14.x[i, j, k] for i in problem_14.I for k in problem_14.K) / problem_14.CuanMad[j] >= problem_14.Dem[j]
    )

problem_14.pre1 = ConstraintList()
for i in problem_14.I:
    problem_14.pre1.add(sum(problem_14.x[i, j, 1] for j in problem_14.J) <= 100 * problem_14.y[i, 1])

problem_14.pre2 = ConstraintList()
for i in problem_14.I:
    problem_14.pre2.add(sum(problem_14.x[i, j, 2] for j in problem_14.J) >= 101 * problem_14.y[i, 2])

problem_14.pre3 = ConstraintList()
for i in problem_14.I:
    problem_14.pre3.add(sum(problem_14.x[i, j, 2] for j in problem_14.J) <= 500 * problem_14.y[i, 2])

problem_14.soloUno = ConstraintList()
for i in problem_14.I:
    problem_14.soloUno.add(sum(problem_14.y[i, k] for k in problem_14.K) <= 1)

# solve
solver = SolverFactory('gurobi')
solver.solve(problem_14)

# print results
print('Objective function value:')
print(problem_14.obj())
for i in problem_14.I:
    for j in problem_14.J:
        for k in problem_14.K:
            print('x[', i, ',', j, ',', k, ']=', problem_14.x[i, j, k].value)



Objective function value:
4060.0
x[ 1 , 1 , 1 ]= -0.0
x[ 1 , 1 , 2 ]= 334.0
x[ 1 , 2 , 1 ]= -0.0
x[ 1 , 2 , 2 ]= 18.0
x[ 1 , 3 , 1 ]= -0.0
x[ 1 , 3 , 2 ]= -0.0
x[ 2 , 1 , 1 ]= -0.0
x[ 2 , 1 , 2 ]= 0.0
x[ 2 , 2 , 1 ]= -0.0
x[ 2 , 2 , 2 ]= 500.0
x[ 2 , 3 , 1 ]= -0.0
x[ 2 , 3 , 2 ]= -0.0
x[ 3 , 1 , 1 ]= -0.0
x[ 3 , 1 , 2 ]= -0.0
x[ 3 , 2 , 1 ]= 16.0
x[ 3 , 2 , 2 ]= -0.0
x[ 3 , 3 , 1 ]= 84.0
x[ 3 , 3 , 2 ]= -0.0


In [57]:
problem_15 = ConcreteModel()

# Define the sets
problem_15.I = RangeSet(1, 20)
problem_15.Tmax = Param(initialize=40)
problem_15.T = RangeSet(1, problem_15.Tmax)
problem_15.FIN = Set(initialize=[17, 18, 19, 20])
problem_15.SUCE = Set(initialize=[(6, 1), (6, 2), (7, 2), (7, 3), (8, 3), (8, 4),
                                  (9, 4), (9, 5), (10, 6), (11, 6), (11, 7), (12, 8),
                                  (12, 9), (13, 9), (14, 11), (15, 14), (15, 16),
                                  (16, 12), (17, 14), (18, 14), (18, 15), (19, 15),
                                  (19, 16), (20, 16)])

demand = {1: 2,
          2: 1,
          3: 3,
          4: 1,
          5: 2,
          6: 1,
          7: 3,
          8: 1,
          9: 2,
          10: 3,
          11: 1,
          12: 3,
          13: 2,
          14: 3,
          15: 2,
          16: 3,
          17: 2,
          18: 1,
          19: 1,
          20: 3}

problem_15.D = Param(problem_15.I, initialize=demand)
problem_15.M = Param(initialize=100000)

# Define the variables
problem_15.x = Var(problem_15.I, problem_15.T, within=Binary)
problem_15.horas = Var(within=NonNegativeReals)

# Define the objective function
problem_15.obj = Objective(
    expr=sum(problem_15.x[i, t] * problem_15.D[i] for i in problem_15.I for t in problem_15.T) + problem_15.horas
)

# Define the constraints
problem_15.r1 = ConstraintList()
for i in problem_15.I:
    problem_15.r1.add(sum(problem_15.x[i, t] for t in problem_15.T) <= 1)

problem_15.r2 = Constraint(expr=sum(problem_15.x[i, t] for i in problem_15.FIN for t in problem_15.T) >= 1)

problem_15.r3 = ConstraintList()
for i, j in problem_15.SUCE:
    problem_15.r3.add(
        sum(problem_15.x[j, t] * (t + problem_15.D[j]) for t in problem_15.T) <=
        problem_15.M * (1 - sum(problem_15.x[i, t] for t in problem_15.T)) +
        sum(problem_15.x[i, t] * t for t in problem_15.T)
    )

problem_15.r4 = ConstraintList()
for i, j in problem_15.SUCE:
    problem_15.r4.add(sum(problem_15.x[j, t] for t in problem_15.T) >= sum(problem_15.x[i, t] for t in problem_15.T))

problem_15.r5 = Constraint(expr=problem_15.horas == sum(problem_15.x[i, t] * (t + problem_15.D[i]) for i in problem_15.FIN for t in problem_15.T))

# solve
solver = SolverFactory('gurobi')
solver.solve(problem_15)

# print results
print('Objective function value:')
print(problem_15.obj())
for i in problem_15.I:
    for t in problem_15.T:
        if problem_15.x[i, t].value > 0:
            print('x[', i, ',', t, ']=', problem_15.x[i, t].value)
print('horas=', problem_15.horas.value)



Objective function value:
29.0
x[ 1 , 1 ]= 1.0
x[ 2 , 1 ]= 1.0
x[ 3 , 1 ]= 1.0
x[ 6 , 3 ]= 1.0
x[ 7 , 4 ]= 1.0
x[ 11 , 7 ]= 1.0
x[ 14 , 8 ]= 1.0
x[ 17 , 11 ]= 1.0
horas= 13.0
